In [22]:
import age
import age.LoadStoreAge.LoadAge as load_age, age.LoadStoreAge.StoreAge as store_age
GRAPH_NAME = "test_csv_load_age"
DSN = "host=127.0.0.1 port=5432 dbname=postgres user=postgres password=password"
# connect to postgreSQL database using AGE extension
age_obj = age.Age()
ag = age_obj.connect(graph=GRAPH_NAME, dsn= DSN)

Create test nodes and edges, with simple properties here

In [23]:
ag.execCypher('MATCH (u) DETACH DELETE u')
# create test nodes, edges here
total_edges = 4000
for i in range(total_edges):
    ag.execCypher('CREATE (n: VertexA {id: %s})-[e: EDGE {id: %s}]->(m: VertexB {id: %s})', params=(str(i), str(i), str(i)))


ag.commit()

Load graph vertices, edges into csv

-> provide absolute paths wherever in the function call path is provided

In [25]:
store_age.load_labels_into_file(ag, '/path_to/age/drivers/python/samples/age_load/data/')

In [27]:
store_age.load_edges_into_file(ag, '/path_to/age/drivers/python/samples/age_load/data/')

Delete the already created vertices, edges to check loading of graph from csv

In [28]:
ag.execCypher('MATCH (u) DETACH DELETE u')
ag.commit()

Load vertices, edges from csv

In [29]:
load_age.load_labels_from_file(ag, '/path_to/age/drivers/python/samples/age_load/data/VertexA.csv', 'VertexA')

In [30]:
load_age.load_labels_from_file(ag, '/path_to/age/drivers/python/samples/age_load/data/VertexB.csv', 'VertexB')

In [31]:
load_age.load_edges_from_file(ag, '/path_to/age/drivers/python/samples/age_load/data/EDGE.csv', 'EDGE')

Create test nodes, edges with nested properties -- in case any of the nodes, edge has a nested property, graph is stored in json

In [32]:
ag.execCypher('MATCH (u) DETACH DELETE u')
# create test nodes, edges here
total_edges = 4000
for i in range(total_edges):
    ag.execCypher('''
                  CREATE (n: NodeA {key1: {
        subkey1: {
            subsubkey1: %s
        }
    }
    })-[e: EDGE {id: %s,key1: {
        subkey1: {
            subsubkey1: %s
        }
    }}]->(m: NodeB {key3: {
        subkey1: {
            subsubkey1: %s
        }}})
    RETURN n''', params = (str(i),str(i),str(i),str(i),))


ag.commit()

Load nodes, edges into the json

In [33]:
store_age.load_labels_into_file(ag, '/path_to/age/drivers/python/samples/age_load/data/')

In [34]:
store_age.load_edges_into_file(ag, '/path_to/age/drivers/python/samples/age_load/data/')

Delete the created nodes to test the loading of the nodes, edges from json

In [35]:
ag.execCypher('MATCH (u) DETACH DELETE u')
ag.commit()

Load nodes, edges from json 

In [36]:
load_age.load_labels_from_file(ag, '/path_to/age/drivers/python/samples/age_load/data/NodeA.json', 'NodeA')

In [37]:
load_age.load_labels_from_file(ag, '/path_to/age/drivers/python/samples/age_load/data/NodeB.json', 'NodeB')

In [38]:
load_age.load_edges_from_file(ag, '/path_to/age/drivers/python/samples/age_load/data/EDGE.json', 'EDGE')

In [18]:
ag.close()